# SETUP

In [11]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import psycopg2
from sqlalchemy import create_engine

# GET DATA

In [3]:
# fetch a web page
htlm_page = requests.get("https://www.francetvinfo.fr/sports/")
print(htlm_page.text[:1000])

<!DOCTYPE html><!--[if lt IE 7]><html class="no-js ie ie6 lte9 lte8 lte7" lang="fr"> <![endif]--><!--[if IE 7]><html class="no-js ie ie7 lte9 lte8 lte7" lang="fr"> <![endif]--><!--[if IE 8]><html class="no-js ie ie8 lte9 lte8" lang="fr"> <![endif]--><!--[if IE 9]><html class="no-js ie ie9 lte9" lang="fr"> <![endif]--><!--[if gt IE 9]><html class="no-js ie" lang="fr"> <![endif]--><!--[if !IE]><!--><html class="no-js" lang="fr"> <!--<![endif]--><head><title>Sports</title><meta http-equiv="pragma" content="no-cache" /><meta http-equiv="expires" content="Tue, 22 Feb 2000 12:14:57 GMT" /><meta http-equiv="refresh" content="1799" /><meta property="og:title" content="Sports" /><meta property="og:description" content="Suivez en direct toute l'actualité &quot;Sports&quot; : vivez l'info en live, en images et en vidéos. Participez, commentez et partager avec Franceinfo en temps réel !" /><meta property="og:type" content="website" /><meta property="og:locale:alternate" content="fr_FR" /><meta pro

In [4]:
page = BeautifulSoup(htlm_page.text, "html5lib")
print(page.get_text()[:1000])

Sports  var myHash='',queryString=document.location.search;var xtref=document.referrer.replace(/[<>]/g,'').replace(/&/g,'$');myHash='xtref='+((''!=xtref)?xtref:'acc_dir');if(document.location.search&&(new RegExp("xtor=([^&]+)")).test(document.location.search)){var xtor=(new RegExp("xtor=([^&]+)")).exec(document.location.search);if(xtor){myHash=((myHash!='')?'&':'')+'xtor='+xtor[1]} console.log('document.location.search',document.location.search);var queryString=document.location.search.replace(xtor[0],'');console.log("queryString",queryString);} myHash=(document.location.hash!='')?document.location.hash+((myHash!='')?'&'+myHash:''):'#'+myHash;(function(mobileUri){var isIpad=navigator.userAgent.match(/iPad/i)!=null,isMobile="undefined"!==typeof window.screen&&"undefined"!==typeof window.screen.width&&(768>=window.screen.width);if(isMobile&&!isIpad){window.location.href="//"+window.location.hostname.replace("www","mobile")+mobileUri}})(("/sports/"+queryString+myHash));  (function(){("und

## EXTRACT TITLES

In [5]:
# extract title
summaries = page.find_all('div', class_="flowItem")

In [6]:
summaries[0]

<div class="flowItem feature" entity="CMS-Audio_3808821"><a href="/replay-radio/en-route-vers-paris-2024/en-route-vers-paris-2024-le-sport-mene-a-tout_3808821.html"><figure id="image_3623465"><span class="audioIcon"></span>    <img alt="L\'escrimeuse Astrid Guyart (à gauche), aux côtés de Cécilia Berder, à l\'occasion de la sortie de son 5ème livre jeunesse \&quot;JO, tout feu, tout glace\&quot;." botag="CMS-IMAGE_3623465" class="b-lazy" data-src="/image/75rukpa5z-9567/840/472/20939301.jpg" height="472px" src="data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==" title="L'escrimeuse Astrid Guyart (à gauche), aux côtés de Cécilia Berder, à l'occasion de la sortie de son 5ème livre jeunesse &quot;JO, tout feu, tout glace&quot;. | FABRICE RIGOBERT / RADIO FRANCE" width="840px"/></figure><div class="right-content"><h2 class="title" field="title">En route vers Paris 2024. Le sport mène à tout</h2><div class="description" field="description">Cécilia Berder, membre de l’

In [7]:
summaries[0].select_one("h2").get_text()

'En route vers Paris 2024. Le sport mène à tout'

In [8]:
len(summaries)

28

In [9]:
sport_news = [x.select_one("h2").get_text() for x in summaries]

In [10]:
sport_news

['En route vers Paris 2024. Le sport mène à tout',
 "Ligue 1 : au terme d'un match fou, le Paris Saint-Germain se saborde à Amiens (4-4)",
 "Foot : une bagarre éclate entre supporters, en marge d'une rencontre entre Sedan et à Bastia",
 'VIDEO. Le Suédois Armand Duplantis bat de nouveau le record du monde de la perche, en franchissant 6,18 mètres',
 "Handball : ce que l'on sait sur l'affaire des tests de grossesse qui secoue un club nantais",
 'Argentine : huit rugbymen incarcérés après avoir battu à mort un jeune homme de 18 ans',
 'VIDEO. Isabelle Sarech, patineuse artistique violée à 16 ans : "Il a fallu #MeToo pour que je réalise"',
 "Abus sexuels dans le sport : quatre plaintes visent des encadrants et un président de club d'escalade",
 'Fair-play financier : Manchester City exclu de toutes les compétitions européennes pendant deux ans',
 "Handball : des tests de grossesse réalisés à l'insu des joueuses ?",
 'Tests de grossesse illégaux à Nantes : le président du club de handball 

# WRITE THE DATA IN POSTGRESQL DB

In [29]:
df = pd.DataFrame({"source": "franceinfo", "headline": sport_news, "date": pd.to_datetime('today').to_pydatetime().date()})

In [30]:
df.head()

,source,headline,date
0,franceinfo,En route vers Paris 2024. Le sport mène à tout,2020-02-16
1,franceinfo,"Ligue 1 : au terme d'un match fou, le Paris Sa...",2020-02-16
2,franceinfo,"Foot : une bagarre éclate entre supporters, en...",2020-02-16
3,franceinfo,VIDEO. Le Suédois Armand Duplantis bat de nouv...,2020-02-16
4,franceinfo,Handball : ce que l'on sait sur l'affaire des ...,2020-02-16


In [31]:
# create an engine
alchemy_engine = create_engine('postgresql+psycopg2://antoinemertz@localhost/news', pool_recycle=3600)

In [32]:
conn = alchemy_engine.connect()
table = "sport"

In [33]:
try:
    frame = df.to_sql(name=table, con=conn, if_exists='append', index=False)
except Exception as ex:
    print(ex)
conn.close()